In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, uniform
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams['figure.figsize'] = (15,10)

In [2]:
threshold_ = np.load('thres3.npy')
upper_, lower_ = threshold_[0][:96], threshold_[1][:96]

In [3]:
start_cost = 3 * 1000 * 16 * 7.6
r = 0.05

def R(ss_t):
    return (ss_t - 5) * 1000 * 16 * 7.6

def Profit(s0, ss0, ss1, upper, lower):
    idx_off = (s0 == 0)
    idx_on = (s0 == 1)

    turn_on = idx_off * (ss0 > upper)
    turn_off = idx_on * (ss0 < lower)

    profit = R(ss1)
    profit[turn_on] = - start_cost
    profit[idx_off & (ss0 <= upper)] = 0

    action = idx_off * (ss0 > upper) + idx_on * (ss0 >= lower)

    return profit, action #action=s1

def valuation(power, gas, upper, lower):#power & gas(10000*97);upper & lower(96)
    ss = (power - 12 * gas)
    n = ss.shape[0]
    N = 2 * 12 * 4 #96
    s = np.zeros((n,N+1))
    V = 0
    for i in range(N): #0-95
        profit, s[:, i+1] = Profit(s[:,i], ss[:,i], ss[:,i+1], upper[i], lower[i])
        V += profit * np.exp(-r*(i+1)/48)
    return np.mean(V)

In [62]:
def new_path(eval_para_pos, n = 5000):
#     N_month = 97
    N = 24
    N *= 4
    dt = 1/48
    thetap1 = [39.28, 40.65, 40.48, 42.75, 46.62, 71.88, 86.92, 103.63, 60.78, 35.55, 39.67, 44.47]
    thetag1 = [2.51, 2.49, 2.66, 2.59, 5.28, 6.33, 6.39, 5.93, 3.12, 2.73, 2.02, 3.07]
    p_theta_std = np.array([1.45, 1.92, 2.36, 2.65, 5.01, 8.66, 12.21, 13.79, 12.78, 14.01, 12.77, 10.9])
    g_theta_std = np.array([0.17, 0.2, 0.28, 0.4, 0.43, 0.67, 0.48, 0.53, 0.45, 0.78, 0.84, 0.71])
    variable_list = [7, 20, 3, 75, 0.083, 35, 0, 3, 75, 7.22, 0.3, 28.87, 10.83, np.array(thetap1), np.array(thetag1)]
    variables_names = ['alphaCC', 'alphaJC', 'alphaG', 
                       'm', 'p', 'CC0', 'JC0', 'G0',
                       'spike_thres', 'sigmaG', 'rho', 
                       'v_summer', 'v_winter', 'PowerTheta', 'GasTheta']
    vec_res_p = list()
    vec_res_g = list()
    month_list = [i for i in range(1, 98)]
    num_ses = np.arange(-3, 3.2, 0.2)
    len_muls = len(num_ses)
    result_ = list()    
    sensitivity_matrix = np.zeros((len(num_ses), len(num_ses)))

    V = np.zeros((N+1,n))
    W = norm.rvs(size = (N+1,n))*np.sqrt(dt)
    Wtilde = norm.rvs(size = (N+1,n))*np.sqrt(dt)
    
    
    for h in range(len(num_ses)):
        for j in range(len(num_ses)):

            if eval_para_pos[0] == 0:
                new_variable = 7 - num_ses[h] * 0.5
            if eval_para_pos[1] == 0:
                new_variable2 = 7 - num_ses[j] * 0.5
                
            if eval_para_pos[0] == 1:
                new_variable = 20 - num_ses[h] * 1.3
            if eval_para_pos[1] == 1:
                new_variable2 = 20 - num_ses[j] * 1.3
                 
            if eval_para_pos[0] == 2:
                new_variable = 3 - num_ses[h] * 0.2
            if eval_para_pos[1] == 2:
                new_variable2 = 3 - num_ses[j] * 0.2                

            if eval_para_pos[0] == 3:
                new_variable = 75 - num_ses[h] * 5
            if eval_para_pos[1] == 3:
                new_variable2 = 75 - num_ses[j] * 5
            
            if eval_para_pos[0] == 4:
                new_variable = 0.083 - num_ses[h] * 0.005
            if eval_para_pos[1] == 4:
                new_variable2 = 0.083 - num_ses[j] * 0.005
                
            if eval_para_pos[0] == 9:
                new_variable = 7.22 - num_ses[h] * 0.05  
            if eval_para_pos[1] == 9:
                new_variable2 = 7.22 - num_ses[j] * 0.05  
                
                
            if eval_para_pos[0] == 10:
                new_variable = 0.3 - num_ses[h] * 0.05
            if eval_para_pos[1] == 10:
                new_variable2 = 0.3 - num_ses[j] * 0.05
                
            if eval_para_pos[0] == 11:
                new_variable = 28.87 - num_ses[h] * 2
            if eval_para_pos[1] == 11:
                new_variable2 = 28.87 - num_ses[j] * 2
                
            if eval_para_pos[0] == 12:
                new_variable = 10.83 - num_ses[h] * 2
            if eval_para_pos[1] == 12:
                new_variable2 = 10.83 - num_ses[j] * 2
                
            if eval_para_pos[0] == 13:
                new_variable = variable_list[13] - num_ses[h] * p_theta_std
            if eval_para_pos[1] == 13:
                new_variable2 = variable_list[13] - num_ses[j] * p_theta_std                

            if eval_para_pos[0] == 14:
                new_variable = variable_list[14] - num_ses[h] * g_theta_std
            if eval_para_pos[1] == 14:
                new_variable2 = variable_list[14] - num_ses[j] * g_theta_std
#             else:
#                 print('Wrong input of eval_para_pos')

            variable_list[eval_para_pos[0]] = new_variable
            variable_list[eval_para_pos[1]] = new_variable2

            alphaCC  = variable_list[0]
            alphaJC  = variable_list[1]
            alphaG  = variable_list[2]
            m  = variable_list[3]
            p  = variable_list[4]
            CC0  = variable_list[5]
            JC0  = variable_list[6]
            G0  = variable_list[7]
            spike_thres  = variable_list[8]
            sigmaG  = variable_list[9]/np.sqrt(dt)/100
            rho  = variable_list[10]
            v_summer  = variable_list[11]/np.sqrt(dt)/100
            v_winter  = variable_list[12]/np.sqrt(dt)/100
            PowerTheta  = variable_list[13]
            GasTheta  = variable_list[14]


#             V = np.zeros((N+1,n))
#             W = norm.rvs(size = (N+1,n))*np.sqrt(dt)
#             Wtilde = norm.rvs(size = (N+1,n))*np.sqrt(dt)
#             B = rho*W + np.sqrt(1-rho**2)*Wtilde
            B = rho*W + np.sqrt(1-rho**2)*Wtilde

            CC = np.zeros((N+1,n)) 
            CC[0,:] = CC0
            JC = np.zeros((N+1,n))
            JC[0,:] = JC0
            G = np.zeros((N+1,n)) 
            G[0,:] = G0
            PC = np.zeros((N+1,n))
            PC[0,:] = CC[0,:]


            for i in range(1, N + 1):
                month1 = month_list[int(np.ceil(i/4)) - 1] if np.ceil(i/4) <= 12 else month_list[int(np.ceil(i/4)) - 1-12]
                monthIndicator = (month1 > 4)&(month1 < 9)

                V[i,:] = monthIndicator*v_summer + (1 - monthIndicator)*v_winter
                CC[i,:] = alphaCC*(thetap1[month1-1] - CC[i-1,:])*dt + V[i,:]*CC[i-1,:]*W[i,:] + CC[i-1,:]
                JC[i,:] = alphaJC*( 0 - JC[i-1,:])*dt + m*(uniform.rvs() < p)+JC[i-1,:]
                #Power Price
                PC[i,:] = CC[i,:] + JC[i,:]*(PC[i-1,:] > spike_thres)
                #Gas Price
                G[i,:]  = alphaG*(thetag1[month1-1] - G[i-1,:])*dt + sigmaG * G[i-1,:] * B[i,:] + G[i-1,:]   
            sensitivity_matrix[h, j] = valuation(PC.T, G.T, upper_, lower_)
    return sensitivity_matrix
#             result_.append(valuation(PC.T, G.T, upper_, lower_))
#     results = np.array(result_) / result_[int(len_muls / 2)]
    
#     return (variables_names[eval_para_pos], np.mean(np.abs(results - 1)), np.std(np.abs(results - 1)), result_[int(len_muls / 2)])
#     plt.plot(num_ses, results, 'o')
#     plt.title('Final Profit Sensitivity Analysis - {}'.format(variables_names[eval_para_pos]))
#     plt.xlabel('multiplier of SE')
#     plt.ylabel('Profit')
    
#     return PC, G

In [73]:
import plotly
import plotly.graph_objects as go
def plotly_surface(df_temp, eval_para_pos):
    
    variables_names = ['alphaCC', 'alphaJC', 'alphaG', 
                       'm', 'p', 'CC0', 'JC0', 'G0',
                       'spike_thres', 'sigmaG', 'rho', 
                       'v_summer', 'v_winter', 'PowerTheta', 'GasTheta']

    x_name, y_name = variables_names[eval_para_pos[1]], variables_names[eval_para_pos[0]]
    
    fig = go.Figure(data=[go.Surface(z=df_temp.values,
                                     y=df_temp.index.values, 
                                     x=df_temp.columns.values[::-1], 
                                     colorbar={"title": "Number", "len": 0.5, "thickness": 15},
                                     colorscale="YlGnBu", 
                                     opacity=0.8)],
                      layout=go.Layout(
#                                       title='Interpolated Volatility Surface', 
                                       autosize=True, height=800,
                                       scene={"xaxis": {'title': 'Multiplier of SE added on {}'.format(x_name),
                                              "tickfont": {"size": 10}, 'type': "linear"},
                                              "yaxis": {"title": 'Multiplier of SE added on {}'.format(y_name), "tickfont": {"size": 10}, "tickangle": 1},
                                              "zaxis": {"title": 'valuation', "tickfont": {"size": 10}},
                                              "camera": {"eye": {"x": 2, "y": 1, "z": 1.25}}, "aspectmode": "cube"}))
#     fig.show()
    return fig

In [ ]:
'Multiplier of SE added on '

In [192]:
s_m = new_path((13, 14), 20000)

In [24]:
s_m_df = pd.DataFrame(s_m, index=np.arange(-3, 3.2, 0.2), columns=np.arange(-3, 3.2, 0.2))

In [47]:
plotly_surface(s_m_df, (13, 14))

In [195]:
s_m_2 = new_path((11, 12), 20000)

In [54]:
s_m_df2 = pd.DataFrame(s_m_2, index=np.arange(-3, 3.2, 0.2), columns=np.arange(-3, 3.2, 0.2))

In [55]:
plotly_surface(s_m_df2, (13, 14))

In [67]:
s_m_3 = new_path((3, 4), 40000)
s_m_df3 = pd.DataFrame(s_m_2, index=np.arange(-3, 3.2, 0.2), columns=np.arange(-3, 3.2, 0.2))

In [70]:
fig3 = plotly_surface(s_m_df3, (3, 4))

In [231]:
fig3

In [74]:
plotly.offline.plot(fig3, filename='test.html')

'test.html'

In [75]:
fig2 = plotly_surface(s_m_df2, (13, 14))

In [76]:
plotly.offline.plot(fig2, filename='test.html')

'test.html'

In [77]:
help(plotly.io.write_html)

Help on function write_html in module plotly.io._html:

write_html(fig, file, config=None, auto_play=True, include_plotlyjs=True, include_mathjax=False, post_script=None, full_html=True, animation_opts=None, validate=True, default_width='100%', default_height='100%', auto_open=False)
    Write a figure to an HTML file representation
    
    Parameters
    ----------
    fig:
        Figure object or dict representing a figure
    file: str or writeable
        A string representing a local file path or a writeable object
        (e.g. an open file descriptor)
    config: dict or None (default None)
        Plotly.js figure config options
    auto_play: bool (default=True)
        Whether to automatically start the animation sequence on page load
        if the figure contains frames. Has no effect if the figure does not
        contain frames.
    include_plotlyjs: bool or string (default True)
        Specifies how the plotly.js library is included/loaded in the output
        div st

In [83]:
plotly.io.write_html(fig2, file='test.html', full_html=False)

In [82]:
plotly.io.write_html(fig3, file='test.html', full_html=False)

In [84]:
fig2.append(fig3)

AttributeError: 'Figure' object has no attribute 'append'

In [110]:
from scipy.ndimage.filters import gaussian_filter
from scipy.signal import medfilt

In [290]:
test_3 = pd.DataFrame(gaussian_filter(s_m_df3, sigma=4), index=-np.arange(-3, 3.2, 0.2), columns=-np.arange(-3, 3.2, 0.2)) 

In [291]:
fig_test_3 = plotly_surface(test_3, (3, 4))

In [283]:
fig_test_3

In [168]:
def mean_filter(matrix, size):
    
    w, l = matrix.shape
    res = np.zeros((w, l))
    for i in range(w):
        for j in range(l):
            if i < int(size / 2):
                r_start = 0
                r_end = size
            elif (w - i) < int(size / 2):
                r_start = w - size
                r_end = w
            else:
                r_start = i - int(size / 2)
                r_end = i + int(size / 2) + 1

                
            if j < int(size / 2):
                c_start = 0
                c_end = size
            elif (l - j) < int(size / 2):
                c_start = l - size
                c_end = l
            else:
                c_start = j - int(size / 2)
                c_end = j + int(size / 2) + 1
#             print(j, l - j, int(size/2))
#             print(c_start, c_end)
                
            
            mean_ = matrix[r_start:r_end, c_start:c_end].mean()
            res[i, j] = mean_
    return res

In [217]:
fig_test = plotly_surface(test, (3, 4))

In [243]:
fig_test.show()

In [221]:
fig_test_3.show()

In [193]:
test_s_m1 = pd.DataFrame(gaussian_filter(s_m, sigma=7), index=np.arange(-3, 3.2, 0.2), columns=np.arange(-3, 3.2, 0.2)) 
fig_test_s_m1 = plotly_surface(test_s_m1, (13, 14))

In [194]:
fig_test_s_m1.show()

In [239]:
test_s_m2 = pd.DataFrame(gaussian_filter(s_m_2, sigma=5), index=np.arange(-3, 3.2, 0.2), columns=np.arange(-3, 3.2, 0.2)) 
fig_test_s_m2 = plotly_surface(test_s_m2, (11, 12))

In [240]:
fig_test_s_m2.show()

In [200]:
s_m_4 = new_path((9, 10), 20000)

In [205]:
test_s_m4 = pd.DataFrame(gaussian_filter(s_m_4, sigma=5), index=np.arange(-3, 3.2, 0.2), columns=np.arange(-3, 3.2, 0.2)) 
fig_test_s_m4 = plotly_surface(test_s_m4, (9, 10))

In [207]:
fig_test_s_m4.show()

In [208]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
def plotly_subplots(n, m, df_list):
    rows = n 
    cols = m
    specs_ = list()
    shapes = list()
    sub_shapes = {'xaxis_title':'X AXIS TITLE', 
                  'yaxis_title':'Y AXIS TITLE', 
                  'zaxis_title':'Z AXIS TITLE'}
    for i in range(n):
        sub_specs = list()
        for j in range(m):
            sub_specs.append({'type': 'surface'})
            shapes.append(sub_shapes)
        specs_.append(sub_specs)
    fig = make_subplots(rows=n, cols=m, specs=specs_)
    ix = 0
    for i in range(n):
        for j in range(m):
            df_temp = df_list[ix]
            fig.add_trace(
                         go.Surface(x=df_temp.columns.values, 
                                    y=df_temp.index.values, 
                                    z=df_temp.values, colorscale='YlGnBu', showscale=False,
                                    opacity=0.8),
                        row=i + 1, col=j + 1)
            z = getattr(fig.layout, "scene{}".format(ix + 1))
            z.update(
                     {"xaxis": {'title': "Num of Nodes in Each Layer","tickfont": {"size": 10}, 'type': "linear"},
                      "yaxis": {"title": "Num of Layers", "tickfont": {"size": 10}, "tickangle": 1},
                      "zaxis": {"title": "Final Loss Value", "tickfont": {"size": 10}},
                      "camera": {"eye": {"x": 2, "y": 1, "z": 1.25}}, "aspectmode": "cube"}
            )
                
            ix += 1
    fig.update_layout(
    title_text='Different Final Losses in 4 Different Training',
    height=1100,
    width=1100)
    iplot(fig)

In [292]:
df_list = [test_s_m1, test_s_m2, test_3, test_s_m4]

In [293]:
x_y_name_list = [(13, 14), (11, 12), (3, 4), (9, 10)]

In [294]:
df_list

[                       -3.0          -2.8          -2.6          -2.4  \
 -3.000000e+00  1.153315e+08  1.153111e+08  1.152711e+08  1.152114e+08   
 -2.800000e+00  1.153057e+08  1.152855e+08  1.152458e+08  1.151866e+08   
 -2.600000e+00  1.152563e+08  1.152364e+08  1.151972e+08  1.151389e+08   
 -2.400000e+00  1.151869e+08  1.151674e+08  1.151291e+08  1.150721e+08   
 -2.200000e+00  1.151024e+08  1.150835e+08  1.150462e+08  1.149909e+08   
 -2.000000e+00  1.150089e+08  1.149906e+08  1.149547e+08  1.149013e+08   
 -1.800000e+00  1.149136e+08  1.148961e+08  1.148615e+08  1.148104e+08   
 -1.600000e+00  1.148235e+08  1.148067e+08  1.147736e+08  1.147248e+08   
 -1.400000e+00  1.147446e+08  1.147285e+08  1.146970e+08  1.146507e+08   
 -1.200000e+00  1.146829e+08  1.146676e+08  1.146376e+08  1.145937e+08   
 -1.000000e+00  1.146429e+08  1.146284e+08  1.145998e+08  1.145585e+08   
 -8.000000e-01  1.146275e+08  1.146136e+08  1.145866e+08  1.145476e+08   
 -6.000000e-01  1.146376e+08  1.146243

In [299]:
def plotly_subplots(n, df_list, x_y_name_list):
    rows = n 
    m = 1
    cols = m
    specs_ = list()
    shapes = list()
    sub_shapes = {'xaxis_title':'X AXIS TITLE', 
                  'yaxis_title':'Y AXIS TITLE', 
                  'zaxis_title':'Z AXIS TITLE'}
    variables_names = ['alphaCC', 'alphaJC', 'alphaG', 
                       'spike_mean', 'p_spike', 'CC0', 'JC0', 'G0',
                       'spike_thres', 'sigmaG', 'rho', 
                       'v_summer', 'v_other', 'PowerTheta', 'GasTheta']
    for i in range(n):
        sub_specs = list()
        for j in range(m):
            sub_specs.append({'type': 'surface'})
            shapes.append(sub_shapes)
        specs_.append(sub_specs)
    fig = make_subplots(rows=n, cols=m, specs=specs_)
    ix = 0
    for i in range(n):
        for j in range(m):
            df_temp = df_list[ix]
            fig.add_trace(
                         go.Surface(x=df_temp.columns.values, 
                                    y=df_temp.index.values, 
                                    z=df_temp.values, colorscale='YlGnBu', showscale=False,
                                    opacity=0.8),
                        row=i + 1, col=j + 1)
            z = getattr(fig.layout, "scene{}".format(ix + 1))
            z.update(
                     {"xaxis": {'title': 'Error term on {}'.format(variables_names[x_y_name_list[i][1]]), 'type': "linear"},
                      "yaxis": {"title": 'Error term added on {}'.format(variables_names[x_y_name_list[i][0]]), "tickfont": {"size": 10}, "tickangle": 1},
                      "zaxis": {"title": "Valuation", "tickfont": {"size": 10}},
                      "camera": {"eye": {"x": 2, "y": 1, "z": 1.25}}, "aspectmode": "cube"}
            )
                
            ix += 1
    fig.update_layout(
    title_text='Sensitivity Analysis',
    height=2000,
    width=2000)
#     iplot(fig)
    return fig

In [300]:
fig_multi = plotly_subplots(4, df_list, x_y_name_list)

In [301]:
plotly.io.write_html(fig_multi, file='test.html', full_html=True)

In [298]:
1

1